# Load Data From CSV

One of the most pervasive formats of data in Machine Learning Land is tabular. Tabular refers to data that is simply stored in a structure of rows and columns that resembles a table.

Tabular data is stored in a format known as COMMA-SEPARATED-VALUES (CSV) which is fairly self explanatory. CSV files are comprised of rows, where each value is separated by commas (,). There are other common separators such as TABS and |.

Let's get started.

## Loading a CSV file

At first, loading a CSV file might seem as simple as:

In [1]:
import scala.io.Source

def naivelyReadCsv(filePath: String): Array[Array[String]] = {
    Source
        .fromFile(filePath)
        .getLines
        .map(_.split(","))
        .toArray
}

val BASE_DATA_PATH = "../../resources/data"
val pimaIndiansPath = s"$BASE_DATA_PATH/1/pima-indians-diabetes.csv"

val data = naivelyReadCsv(pimaIndiansPath)

println(s"Number of rows is ${data.length} and number of columns is ${data.head.length}")

Number of rows is 768 and number of columns is 9


import scala.io.Source


defined function naivelyReadCsv
BASE_DATA_PATH: String = "../../resources/data"
pimaIndiansPath: String = "../../resources/data/1/pima-indians-diabetes.csv"
data: Array[Array[String]] = Array(
  Array("6", "148", "72", "35", "0", "33.6", "0.627", "50", "1"),
  Array("1", "85", "66", "29", "0", "26.6", "0.351", "31", "0"),
  Array("8", "183", "64", "0", "0", "23.3", "0.672", "32", "1"),
  Array("1", "89", "66", "23", "94", "28.1", "0.167", "21", "0"),
  Array("0", "137", "40", "35", "168", "43.1", "2.288", "33", "1"),
  Array("5", "116", "74", "0", "0", "25.6", "0.201", "30", "0"),
  Array("3", "78", "50", "32", "88", "31.0", "0.248", "26", "1"),
  Array("10", "115", "0", "0", "0", "35.3", "0.134", "29", "0"),
  Array("2", "197", "70", "45", "543", "30.5", "0.158", "53", "1"),
  Array("8", "125", "96", "0", "0", "0.0", "0.232", "54", "1"),
  Array("4", "110", "92", "0", "0", "37.6", "0.191", "30", "0"),
...

Fairly good, right? Apparently everything worked out well. 

Sure. But there are many corner cases we didn't consider, such as:

   - Files with headers.
   - Other separators such as | or TABS.
   - Quoted values.
   - Commas inside quoted values.
   - Data types besides Strings.
   - Empty lines.
   
As we can see, our naïve implementation falls short. The true of the matter is that implementing a CSV library from scratch is an unnecessary pain for our purpose, which is getting into the guts of several machine learning algorithms in order to obtain a deeper, better understanding of how they work.

So, that being said, let's rely on [scala-csv](https://github.com/tototoshi/scala-csv) a very, very cool CSV library for Scala.

In [2]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.5`

import $ivy.$                                      

Let's try again using this handy library:

In [3]:
import com.github.tototoshi.csv._

def loadCsvAsString(filePath: String): List[List[String]] = {
    CSVReader.open(filePath).all()
}

val data = loadCsvAsString(pimaIndiansPath)

println(s"Number of rows is ${data.length} and number of columns is ${data.head.length}")

Number of rows is 768 and number of columns is 9


import com.github.tototoshi.csv._


defined function loadCsvAsString
data: List[List[String]] = List(
  List("6", "148", "72", "35", "0", "33.6", "0.627", "50", "1"),
  List("1", "85", "66", "29", "0", "26.6", "0.351", "31", "0"),
  List("8", "183", "64", "0", "0", "23.3", "0.672", "32", "1"),
  List("1", "89", "66", "23", "94", "28.1", "0.167", "21", "0"),
  List("0", "137", "40", "35", "168", "43.1", "2.288", "33", "1"),
  List("5", "116", "74", "0", "0", "25.6", "0.201", "30", "0"),
  List("3", "78", "50", "32", "88", "31.0", "0.248", "26", "1"),
  List("10", "115", "0", "0", "0", "35.3", "0.134", "29", "0"),
  List("2", "197", "70", "45", "543", "30.5", "0.158", "53", "1"),
  List("8", "125", "96", "0", "0", "0.0", "0.232", "54", "1"),
  List("4", "110", "92", "0", "0", "37.6", "0.191", "30", "0"),
...

Good, things still work! The main difference between our naïve implementation and this is that scala-csv opts for returning a list of lists, which is a data structure less performant than arrays for random access of elements.

I know, I know... Not too impressive. The good thing about this library is that we can define several parameters such as separators, quoted values and such for our reader like this:

```
implicit object MyFormat extends DefaultCSVFormat {
  override val delimiter = '#'
}
```

If you want to know more about scala-csv and how to tweak it, just go to their GitHub page :)

## Converting Strings to Doubles

We can see that all of our data in the `pima-indians-diabetes.csv` dataset is numeric, so it doesn't really makes sense to have them stored as strings. If we think about it, there are only two possible data types we are interested in: Text and Numeric. For convenience, let's choose Double as our default numeric value. Let's create a Data trait to represent this:

In [4]:
sealed trait Data

case class Numeric(value: Double) extends Data
case class Text(value: String) extends Data

defined trait Data
defined class Numeric
defined class Text

In [5]:
def loadCsv(filePath: String): Vector[Vector[Data]] = {
    val reader = CSVReader.open(filePath)
    
    reader
        .toStream
        .map(x => x.toArray.map(Text(_)).toVector)
        .toVector
}

val data = loadCsv(pimaIndiansPath)

println(s"Number of rows is ${data.length} and number of columns is ${data.head.length}")

Number of rows is 768 and number of columns is 9


defined function loadCsv
data: Vector[Vector[Data]] = Vector(
  Vector(
    Text(6),
    Text(148),
    Text(72),
    Text(35),
    Text(0),
    Text(33.6),
    Text(0.627),
    Text(50),
    Text(1)
  ),
...

Now that we abstracted the data, we can create a function to convert text columns to numeric columns.

In [6]:
def textColumnToNumeric(data: Vector[Vector[Data]], columnIndex: Int) = {
    data.map { row => 
        val (firstHalf, secondHalf) = row.splitAt(columnIndex)
        val affectedValue = 
            secondHalf.head match { 
                case Text(value) => Numeric(value.toDouble) 
                case d => d
            }
    
        firstHalf ++ Vector(affectedValue) ++ secondHalf.tail
    }
}

textColumnToNumeric(data, 0).take(5).foreach(println)

Vector(Numeric(6.0), Text(148), Text(72), Text(35), Text(0), Text(33.6), Text(0.627), Text(50), Text(1))
Vector(Numeric(1.0), Text(85), Text(66), Text(29), Text(0), Text(26.6), Text(0.351), Text(31), Text(0))
Vector(Numeric(8.0), Text(183), Text(64), Text(0), Text(0), Text(23.3), Text(0.672), Text(32), Text(1))
Vector(Numeric(1.0), Text(89), Text(66), Text(23), Text(94), Text(28.1), Text(0.167), Text(21), Text(0))
Vector(Numeric(0.0), Text(137), Text(40), Text(35), Text(168), Text(43.1), Text(2.288), Text(33), Text(1))


defined function textColumnToNumeric

As we can see, the first five rows of our dataset have their first column converted to numeric values. If we want to be absolutely sure, let's run the following predicate:

In [7]:
val columnIsNumeric = textColumnToNumeric(data, 0).forall {
    _.head match {
        case _: Numeric => true 
        case _ => false
    }
}

columnIsNumeric: Boolean = true

Great!

## Converting Categories to Numbers

Now we're able to combine text and numeric data in the same data structure. Awesome! 

However, there are some algorithms that doesn't feel quite confortable handling data of diverse types. In fact, they prefer all their data to be numeric. So, for instance, if we have three animal categories such as `Bird`, `Dog`, `Cat`, a model would prefer instead 0, 1 and 2 (or any other three numbers).

Our second dataset (located in `resources/data/1/iris.csv`) has a categorical column. Let's take a look:

In [8]:
val irisDataPath = s"$BASE_DATA_PATH/1/iris.csv"
val irisData = loadCsv(irisDataPath)

println(irisData.head.mkString(", "))

Text(5.1), Text(3.5), Text(1.4), Text(0.2), Text(Iris-setosa)


irisDataPath: String = "../../resources/data/1/iris.csv"
irisData: Vector[Vector[Data]] = Vector(
  Vector(Text(5.1), Text(3.5), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.9), Text(3.0), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.7), Text(3.2), Text(1.3), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.6), Text(3.1), Text(1.5), Text(0.2), Text(Iris-setosa)),
  Vector(Text(5.0), Text(3.6), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(5.4), Text(3.9), Text(1.7), Text(0.4), Text(Iris-setosa)),
  Vector(Text(4.6), Text(3.4), Text(1.4), Text(0.3), Text(Iris-setosa)),
  Vector(Text(5.0), Text(3.4), Text(1.5), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.4), Text(2.9), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.9), Text(3.1), Text(1.5), Text(0.1), Text(Iris-setosa)),
  Vector(Text(5.4), Text(3.7), Text(1.5), Text(0.2), Text(Iris-setosa)),
...

Let's create a function to convert categorical text columns to discrete numeric values.

We will do this by creating a lookup table of categories, where each of them will act as a key and the value will be a unique number that corresponds to that category. In order to revert the transformation, this function will return both the modified dataset and the lookup table.

In [9]:
def categoricalColumnToNumeric(data: Vector[Vector[Data]], columnIndex: Int) = {
    val uniqueColumnValues = data.foldLeft(Set[Data]()) { (set, row) => 
        set + row(columnIndex) 
    }
    
    val lookUpTable = uniqueColumnValues.zipWithIndex.toMap
    
    val categorizedData = data.map { row => 
        val (firstHalf, secondHalf) = row.splitAt(columnIndex)
        val affectedValue = Numeric(lookUpTable(secondHalf.head).toDouble)
    
        firstHalf ++ Vector(affectedValue) ++ secondHalf.tail
    }
    
    (categorizedData, lookUpTable)
}

val (transformedData, lookupTable) = categoricalColumnToNumeric(irisData, 4)
val sampleSize = 10
val sample = scala.util.Random.shuffle(transformedData).take(sampleSize)

sample.foreach(println)

Vector(Text(5.7), Text(3.0), Text(4.2), Text(1.2), Numeric(1.0))
Vector(Text(5.0), Text(3.4), Text(1.5), Text(0.2), Numeric(0.0))
Vector(Text(6.7), Text(3.1), Text(4.4), Text(1.4), Numeric(1.0))
Vector(Text(5.6), Text(2.5), Text(3.9), Text(1.1), Numeric(1.0))
Vector(Text(5.7), Text(2.9), Text(4.2), Text(1.3), Numeric(1.0))
Vector(Text(5.6), Text(2.9), Text(3.6), Text(1.3), Numeric(1.0))
Vector(Text(5.2), Text(2.7), Text(3.9), Text(1.4), Numeric(1.0))
Vector(Text(6.8), Text(2.8), Text(4.8), Text(1.4), Numeric(1.0))
Vector(Text(6.7), Text(3.3), Text(5.7), Text(2.5), Numeric(2.0))
Vector(Text(5.4), Text(3.7), Text(1.5), Text(0.2), Numeric(0.0))


defined function categoricColumnToNumeric
transformedData: Vector[Vector[Data]] = Vector(
  Vector(Text(5.1), Text(3.5), Text(1.4), Text(0.2), Numeric(0.0)),
  Vector(Text(4.9), Text(3.0), Text(1.4), Text(0.2), Numeric(0.0)),
  Vector(Text(4.7), Text(3.2), Text(1.3), Text(0.2), Numeric(0.0)),
  Vector(Text(4.6), Text(3.1), Text(1.5), Text(0.2), Numeric(0.0)),
  Vector(Text(5.0), Text(3.6), Text(1.4), Text(0.2), Numeric(0.0)),
  Vector(Text(5.4), Text(3.9), Text(1.7), Text(0.4), Numeric(0.0)),
  Vector(Text(4.6), Text(3.4), Text(1.4), Text(0.3), Numeric(0.0)),
  Vector(Text(5.0), Text(3.4), Text(1.5), Text(0.2), Numeric(0.0)),
  Vector(Text(4.4), Text(2.9), Text(1.4), Text(0.2), Numeric(0.0)),
  Vector(Text(4.9), Text(3.1), Text(1.5), Text(0.1), Numeric(0.0)),
  Vector(Text(5.4), Text(3.7), Text(1.5), Text(0.2), Numeric(0.0)),
...
lookupTable: Map[Data, Int] = Map(
  Text(Iris-setosa) -> 0,
  Text(Iris-versicolor) -> 1,
  Text(Iris-virginica) -> 2
)
sampleSize: Int = 10
sample: Vector[V

As we can see in the output above, our function created the following mapping table:

```
Map(
  Text(Iris-setosa) -> 0,
  Text(Iris-versicolor) -> 1,
  Text(Iris-virginica) -> 2
)
```

It is possible that different runs of the function associate the categories to different numbers. That's OK. As long as each occurrence of the category gets mapped to the same number, there's no problem ;)
